In [ ]:
# row 데이터 
AI	0	90	20.04
AI	1	45	10.02
AI	2	75	16.7
AI	3	132	29.4
AI	4	12	2.67
AI	5	63	14.03
AI	6	8	1.78
AI	7	13	2.9
AI	8	4	0.89
AI	10	7	1.56

In [ ]:
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
),
category_total AS (
    SELECT category, COUNT(*) AS total FROM career_data GROUP BY category
)

SELECT
    c.category AS 직무,
    c.career AS 연차,
    COUNT(*) AS 채용_공고_개수,
    ROUND(COUNT(*) * 100.0 / ct.total, 2) AS 직무_내_비율
FROM career_data c
JOIN category_total ct ON c.category = ct.category
GROUP BY c.category, c.career, ct.total
ORDER BY c.category, c.career;


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행 (VR 직무 제외)
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
),
category_total AS (
    SELECT category, COUNT(*) AS total FROM career_data GROUP BY category
)

SELECT
    c.category AS 직무,
    c.career AS 연차,
    COUNT(*) AS 채용_공고_개수,
    ROUND(COUNT(*) * 100.0 / ct.total, 2) AS 직무_내_비율
FROM career_data c
JOIN category_total ct ON c.category = ct.category
GROUP BY c.category, c.career, ct.total
ORDER BY c.category, c.career;
"""
cursor.execute(sql_query)
career_data = cursor.fetchall()

# ✅ 직무별 career 데이터 저장
for duty, career, job_count, ratio in career_data:
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, job_count, ratio))  # ✅ 연차, 채용 건수, 비율 저장


# ✅ 모든 직무에 대해 GPT-4 분석 수행
for duty, career_info in job_career_dict.items():
    # ✅ 기존 career 컬럼 값 조회
    cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
    existing_career_data = cursor.fetchone()

    # ✅ 연차별 데이터 변환
    if career_info:
        job_details = "\n".join([
            f"- {'신입사원' if c[0] == '0' else f'{c[0]}년차'}: {c[1]}건"
            for c in career_info
        ])
    else:
        job_details = f"{duty} 직무의 채용 데이터가 없습니다."

    print(f"🎯 {duty} 직무 데이터 확인:\n{job_details}")

    # ✅ GPT-4 프롬프트 생성
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 비율]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 높은 비율을 차지하는 경력 연차와 해당 비율
    2. 두 번째로 높은 비율을 차지하는 경력 연차와 해당 비율
    3. 가장 낮은 비율을 차지하는 경력 연차와 해당 비율
    4. {duty} 직무에서 경력별 채용 비율을 기반으로 한 특징 및 트렌드 분석

    ❗ 단, "경력 0년"은 "신입사원"으로 변환해서 출력해주세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    ❗ "신입사원"이라는 단어가 변형되지 않도록 정확하게 출력해주세요.
    """

    try:
        # ✅ GPT-4 API 호출
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt4_summary = response.choices[0].message.content.strip()

        print(f"✅ {duty} 직무 최종 응답:\n{gpt4_summary}")

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        if existing_career_data and existing_career_data[0]:
            try:
                career_list = ast.literal_eval(existing_career_data[0])  # 문자열 → 리스트 변환
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 기존 이미지 경로 유지
                    career_list = [image_path, gpt4_summary]  # GPT 설명 업데이트
                else:
                    career_list = [gpt4_summary]  # 기존 데이터가 리스트가 아니면 새로 생성
            except:
                career_list = [gpt4_summary]  # 변환 실패 시 GPT 설명만 추가
        else:
            career_list = [gpt4_summary]  # 기존 값이 없으면 새 리스트 생성

        # ✅ 업데이트할 값 변환 (리스트 → 문자열)
        updated_text = str(career_list)

        # ✅ `duty_analysis` 테이블에서 해당 직무의 `career` 값 업데이트
        cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

        print(f"✅ {duty} 직무 career 업데이트 완료: {updated_text}")

    except Exception as e:
        print(f"❌ {duty} 직무 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()


🎯 AI 직무 데이터 확인:
- 0년차: 90건
- 1년차: 45건
- 2년차: 75건
- 3년차: 132건
- 4년차: 12건
- 5년차: 63건
- 6년차: 8건
- 7년차: 13건
- 8년차: 4건
- 10년차: 7건
✅ AI 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 총 공고 수 449건 중 132건으로, 이는 약 29.4%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 신입사원으로, 총 공고 수 449건 중 90건으로 이는 약 20%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 8년차로, 총 공고 수 449건 중 4건으로 이는 약 0.89%입니다.
4. AI 직무에서의 채용 공고 비율을 보면, 3년차 경력자를 가장 선호하는 추세가 보입니다. 다음으로는 신입사원을 많이 뽑는 편입니다. 그러나 4년차 이상으로 넘어갈수록 채용 공고의 비율이 줄어드는 것을 볼 수 있고, 8년차 이상의 경력자는 거의 뽑지 않는 것으로 보입니다. 이는 AI 직무가 계속 변화하고 발전하면서 기술의 최신성을 중요시하는 기업들의 특성이 반영된 결과로 보입니다.
✅ AI 직무 career 업데이트 완료: ['career_charts/AI.png', '1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 총 공고 수 449건 중 132건으로, 이는 약 29.4%입니다.\n2. 두 번째로 높은 비율을 차지하는 경력 연차는 신입사원으로, 총 공고 수 449건 중 90건으로 이는 약 20%입니다.\n3. 가장 낮은 비율을 차지하는 경력 연차는 8년차로, 총 공고 수 449건 중 4건으로 이는 약 0.89%입니다.\n4. AI 직무에서의 채용 공고 비율을 보면, 3년차 경력자를 가장 선호하는 추세가 보입니다. 다음으로는 신입사원을 많이 뽑는 편입니다. 그러나 4년차 이상으로 넘어갈수록 채용 공고의 비율이 줄어드는 것을 볼 수 있고, 8년차 이상의 경력자는 거의 뽑지 않는 것으로 보입니다. 이는 AI 직무가 계속 변화하고 발전하면서 기술의 최신성